## Clone repo and settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks
!cp -f ./milk_classification-main.zip /content/
%cd /content/
!unzip milk_classification-main.zip
!rm -f /content/milk_classification-main.zip

In [ ]:
!git clone https://github.com/duongduc2908/milk_classification.git
%cd milk_classification/
!bash config.sh

In [ ]:
# RESET RUNTIME
!gdown --id 1-U38UAigrcEgzKEZZgjtF-kHnbZfEYbv
!unzip checkpoints.zip
!rm -f checkpoints.zip

In [1]:
%cd /content/milk_classification

/content/milk_classification


## Load model

In [2]:
import re
import ngram
class Analyzer:
    brands = {
        'abbott', 'bellamyorganic', 'blackmores', 'bubs_australia', 'danone', 'f99foods', 'friesland_campina_dutch_lady',
        'gerber', 'glico', 'heinz', 'hipp', 'humana uc', 'mead_johnson', 'megmilksnowbrand', 'meiji', 'morigana',
        'namyang', 'nestle', 'no_brand', 'nutifood', 'nutricare', 'pigeon', 'royal_ausnz', 'vinamilk',
        'vitadairy', 'wakodo'
    }

    def __init__(self, n=3,):
        self.n = n
        self.index = ngram.NGram(N=n)

    def __call__(self, s):
        tokens = re.split(r'\s+', s.lower().strip())
        filtered_tokens = []
        for token in tokens:
            if len(token) > 20:
                continue

            if re.search(r'[?\[\]\(\):!]', token):
                continue

            if re.search(f'\d{2,}', token):
                continue

            filtered_tokens.append(token)

        non_ngram_tokens = []
        ngram_tokens = []

        for token in filtered_tokens:
            if token in self.brands:
                non_ngram_tokens.append(token)
                n_grams = list(self.index.ngrams(self.index.pad(token)))
                ngram_tokens.extend(n_grams)
            else:
                n_grams = list(self.index.ngrams(self.index.pad(token)))
                ngram_tokens.extend(n_grams)
        res = [*non_ngram_tokens, *ngram_tokens]
        return res

In [3]:
from PIL import ImageFont, ImageDraw, Image
import argparse
import sys
import time
from pathlib import Path
import os
import cv2
import torch
import torch.backends.cudnn as cudnn

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, colorstr, non_max_suppression, \
    apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_sync
import numpy as np
from utils.general import xywh2xyxy,clip_coords

os.chdir('textSpotting')
import textSpottingInfer 
os.chdir('..')
os.chdir('classifyText')
import textClassifyInfer
os.chdir('..')
os.chdir('classifyImage')
import objectClasssifyInfer 
os.chdir('..')

craft_detect, model_recognition = textSpottingInfer.load_model_1()
mmocr_recog,pan_detect,classifyModel_level1,mapping_checkpoints = textClassifyInfer.load_model()
chinh_model,model_step,labels_end,labels_branch,dict_middle,dict_step= objectClasssifyInfer.load_model()

with open('/content/milk_classification/keywords.txt', 'r') as f:
    lines = f.readlines()
keywords = []
for line in lines:
    line = line.strip().split()
    keywords += line


model input parameters 32 100 20 1 512 256 130 80 TPS ResNet BiLSTM Attn
loading pretrained model from /content/milk_classification/textSpotting/text-recognition/best_accuracy.pth
Use load_from_http loader


/usr/local/lib/python3.7/dist-packages/mmdet/apis/inference.py:48: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


device: cuda:0


/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Init func utils

In [4]:
from spellchecker import SpellChecker
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline

spell = SpellChecker(language=None,)  # loads default word frequency list
spell.word_frequency.load_text_file('/content/milk_classification/corpus.txt')


def display_image(im_cv):
  im_cv = cv2.cvtColor(im_cv, cv2.COLOR_BGR2RGB)
  pil_img = Image.fromarray(im_cv)
  plt.imshow(pil_img)
  plt.show()

def meger_label_branch(labels,step,name):
    for line in labels:
        words = line.split("/")
        if words[step] == name.lower().replace(" ","_"):
            return words[0]
def word2line(result, img):
    temp = {'center': None, 'text': None}
    new_res = []
    zero_mask = np.zeros(img.shape[:2]).astype('uint8')
    zero_mask_copy = zero_mask.copy()
    for res in result:
        x,y,w,h = cv2.boundingRect(res['boxes'].astype(int))
        zero_mask[y+int(0.3*h):y+int(0.7*h), x:x+w] = 125
        # zero_mask = cv2.polylines(zero_mask, [res['boxes'].astype(int)], True, 255, -1)

        center = np.array([x+0.5*w, y+0.5*h]).astype(int)
        # print(cv2.pointPolygonTest(res['boxes'].astype(int),tuple(center),False))
        item = temp.copy()
        item['center'] = center
        item['text'] = res['text']
        new_res.append(item)

    kernel = np.ones((1, 20), np.uint8)    
    zero_mask = cv2.dilate(zero_mask, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(zero_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # zero_mask_copy = cv2.drawContours(zero_mask_copy, contours, -1, 255, 2)
    # cv2.imrite('mask.jpg', zero_mask_copy)

    temp = {'contour': None, 'text': None, 'box': None}
    final_res = []  
    for contour in contours:
        box = cv2.boundingRect(contour.astype(int))
        item = temp.copy()
        item['box'] = np.array(box)
        item['contour'] = contour

        text_with_center = []
        temp1 = {'center': None, 'text': None}
        for pt in new_res:
            if cv2.pointPolygonTest(contour,tuple(pt['center']),False) > 0:
                item1 = temp1.copy()
                item1['text'] = pt['text']
                item1['center'] = pt['center']
                text_with_center.append(item1)
        
        text_with_center = np.array(text_with_center)
        only_center = [it['center'][0] for it in text_with_center]
        text_with_center = text_with_center[np.argsort(only_center)]
        
        item['text'] = ' '.join([text['text'] for text in text_with_center])
        final_res.append(item)

    return final_res

In [5]:
import json 

class Ensemble():
    
    def __init__(self,branch,result_chinh,result_thanh, json_chinh, json_thanh, text_list):
        self.branch = branch 
        self.result_chinh = result_chinh
        self.result_thanh = result_thanh
        self.text_list = text_list
        self.json_chinh_dict = json_chinh
        self.json_thanh_dict = json_thanh
        # with open(json_chinh,'r') as f:
        #     self.json_chinh_dict =json.load(f)
        # with open(json_thanh,'r') as f:
        #     self.json_thanh_dict =json.load(f)
    
    def run(self,):
        

        result_chinh = self.json_chinh_dict[str(self.result_chinh)]
        if len(result_chinh)>2:
            check_list = True
        else:
            check_list = False
        if len(result_chinh[-1].split('/'))==3:
          branch_chinh,middle_chinh,step_chinh = result_chinh[-1].split('/')
        else:
          branch_chinh,middle_chinh = result_chinh[-1].split('/')
          step_chinh = middle_chinh
        age = -1
        for text in self.text_list:
            if text.isnumeric():
                if int(text)< 10:
                    if age == -1:
                        age = int(text)
                    if age > int(text):
                        age = int(text)
        if age >= 0:
            age = str(age)
        else:
            age = ''
        if self.result_thanh is None:
          if branch_chinh == self.branch:
            if age!='':
              label = self.branch +'/'+middle_chinh +'/'+age
            else:
              label = self.branch +'/' + middle_chinh
            return label
          else:
            return self.branch



        if branch_chinh == self.branch:
            if  age!='': # có tuổi
                label = self.branch + '/' + middle_chinh + '/' + age

            
            else: #không có tuổi
                if not check_list:
                    label = self.branch +'/' + middle_chinh + '/' + step_chinh 
                else: 
                    if self.result_thanh in [i.split('/')[-1] for i in result_chinh]:
                        label = self.branch + '/' + middle_chinh 
                        if middle_chinh=='dutch_baby':
                          label = self.branch + '/' + middle_chinh +'/' + self.result_thanh

                    else:
                        label = self.branch + '/' + middle_chinh 
                    if self.result_thanh=='other':
                        label = self.branch
                    

        else:  # chính khác branch gốc 
            
            if self.result_thanh == 'other':
                label = self.branch 
            else:
                middle_thanh =  self.json_thanh_dict[self.result_thanh].split('/')[0]
                if  age!='': # có tuổi
                    label = self.branch + '/' + middle_thanh + '/' + age
                else: 
                    label = self.branch + '/' + middle_thanh
        if 'nan_optipro' in [i.strip() for i in label.split("/")]:
            if 'optipro' not in [t.lower() for t in self.text_list]:
                label=label.replace('nan_optipro','nan')
        # if 'nan' in [i.strip() for i in label.split("/")]:
        #     if 'optipro' in [t.lower() for t in self.text_list]:
        #         label=label.replace('nan','nan_optipro')
        if 'dielac_grow_plus_blue' in [i.strip() for i in label.split("/")] or 'dielac_grow_plus_red' in [i.strip() for i in label.split("/")]:
            if 'plus' not in self.text_list:
                  label=label.replace('dielac_grow_plus_blue','dielac_grow_blue').replace('dielac_grow_plus_red','dielac_grow_red')
        if 'oggi' in self.text_list or '0ggi' in self.text_list or 'Ogi' in self.text_list :
            label = "vitadary/oggi/" + age
        return label

## **Func main**

In [10]:
def run(weights=['models/weights/binh_new_best.pt', 'models/weights/sua_new_best_2.pt'],  # model.pt path(s)
        # file/dir/URL/glob, 0 for webcam
        source='/content/drive/MyDrive/Colab Notebooks/data_test_end',
        imgsz=640,  # inference size (pixels)
        conf_thres=0.3,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='0',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=True,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        # save results to project/name
        project='/content/drive/MyDrive/Colab Notebooks/results',
        name='data_test_end',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=2,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        ):

    save_img = not nosave and not source.endswith(
        '.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(project) / name,
                              exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True,
                                                          exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    w = weights[0] if isinstance(weights, list) else weights
    classify, suffix = False, Path(w).suffix.lower()
    pt, onnx, tflite, pb, graph_def = (
        suffix == x for x in ['.pt', '.onnx', '.tflite', '.pb', ''])  # backend
    stride, names = 64, [f'class{i}' for i in range(1000)]  # assign defaults
    if pt:
        model_binh_sua = attempt_load(
            weights[0], map_location=device)  # load FP32 model
        model_sua = attempt_load(
            weights[1], map_location=device)  # load FP32 model

        stride_binh = int(model_binh_sua.stride.max())  # model stride
        names_binh = model_binh_sua.module.names if hasattr(
            model_binh_sua, 'module') else model_binh_sua.names  # get class names
        names_sua = model_sua.module.names if hasattr(
            model_sua, 'module') else model_sua.names  # get class names
        if half:
            model_binh_sua.half()  # to FP1
            names_sua.half()  # to FP1
        if classify:  # second-stage classifier
            model_binh_sua_c = load_classifier(
                name='resnet50', n=2)  # initialize
            model_binh_sua_c.load_state_dict(torch.load(
                'resnet50.pt', map_location=device)['model']).to(device).eval()

            model_sua_c = load_classifier(name='resnet50', n=2)  # initialize
            model_sua_c.load_state_dict(torch.load('resnet50.pt', map_location=device)[
                                        'model']).to(device).eval()
    elif onnx:
        check_requirements(('onnx', 'onnxruntime'))
        import onnxruntime
        session = onnxruntime.InferenceSession(w, None)
    imgsz_binh = check_img_size(imgsz, s=stride_binh)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz_binh, stride=stride_binh)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz_binh, stride=stride_binh)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    if pt and device.type != 'cpu':
        model_binh_sua(torch.zeros(1, 3, imgsz_binh, imgsz_binh).to(
            device).type_as(next(model_binh_sua.parameters())))  # run once
        model_sua(torch.zeros(1, 3, imgsz_binh, imgsz_binh).to(
            device).type_as(next(model_sua.parameters())))  # run once
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        if pt:
            img = torch.from_numpy(img).to(device)
            img = img.half() if half else img.float()  # uint8 to fp16/32
        elif onnx:
            img = img.astype('float32')
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if len(img.shape) == 3:
            img = img[None]  # expand for batch dim

        # Inference
        t1 = time_sync()
        if pt:
            visualize = increment_path(
                save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred_binh = model_binh_sua(
                img, augment=augment, visualize=visualize)[0]
            pred_sua = model_sua(img, augment=augment, visualize=visualize)[0]

        # NMS
        pred_binh = non_max_suppression(
            pred_binh, 0.4, iou_thres, classes, agnostic_nms, max_det=max_det)
        pred_sua = non_max_suppression(
            pred_sua, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        t2 = time_sync()

        # Second-stage classifier (optional)
        if classify:
            pred_binh = apply_classifier(
                pred_binh, model_binh_sua_c, img, im0s)
            pred_sua = apply_classifier(pred_sua, model_sua_c, img, im0s)

        # Process predictions binh sua

        for i, (det_binh, det_sua) in enumerate(zip(pred_binh, pred_sua)):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(
                ), dataset.count
            else:
                p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + \
                ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            # normalization gain whwh
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
            imc = im0.copy()  # for save_crop
            with open(txt_path + '.txt', 'a') as f:
                txt_path_end = save_path.replace(
                    '.jpg', '_return.txt').replace('.png', '_return.txt')
                with open(txt_path_end, "w") as file_label:
                    if len(det_binh):
                        # Rescale boxes from img_size to im0 size
                        det_binh[:, :4] = scale_coords(
                            img.shape[2:], det_binh[:, :4], im0.shape).round()

                        # Print results
                        for c in det_binh[:, -1].unique():
                            # detections per class
                            n = (det_binh[:, -1] == c).sum()
                            # add to string
                            s += f"{n} {names_binh[int(c)]}{'s' * (n > 1)}, "

                        # Write results
                        for *xyxy, conf, cls in reversed(det_binh):
                            if save_txt:  # Write to file
                                xywh = (xyxy2xywh(torch.tensor(xyxy).view(
                                    1, 4)) / gn).view(-1).tolist()  # normalized xywh
                                # label format
                                line = (
                                    cls, *xywh, conf) if save_conf else (cls, *xywh)
                                f.write(('%g ' * len(line)).rstrip() %
                                        line + '\n')

                            if save_img or save_crop or view_img:  # Add bbox to image
                                # To classification
                                c = int(cls)  # integer class
                                label = None if hide_labels else (
                                    names_binh[c] if hide_conf else f'{names_binh[c]} {conf:.2f}')
                                im0 = plot_one_box(xyxy, im0, label=label, color=colors(
                                    0, True), line_width=line_thickness)
                                # if save_crop:
                                #     save_one_box(xyxy, imc, file=save_dir / 'crops' / names_binh[c] / f'{p.stem}.jpg', BGR=True)

                    list_bbox = []
                    if len(det_sua):
                        # Rescale boxes from img_size to im0 size
                        det_sua[:, :4] = scale_coords(
                            img.shape[2:], det_sua[:, :4], im0.shape).round()

                        # Print results
                        for c in det_sua[:, -1].unique():
                            # detections per class
                            n = (det_sua[:, -1] == c).sum()
                            # add to string
                            s += f"{n} {names_sua[int(c)]}{'s' * (n > 1)}, "
                        # Write results
                        for *xyxy, conf, cls in reversed(det_sua):
                            list_bbox.append(xyxy)
                            if save_txt:  # Write to file
                                xywh = (xyxy2xywh(torch.tensor(xyxy).view(
                                    1, 4)) / gn).view(-1).tolist()  # normalized xywh
                                # label format
                                line = (
                                    4, *xywh, conf) if save_conf else (4, *xywh)
                                f.write(('%g ' * len(line)).rstrip() %
                                        line + '\n')

                            if save_img or save_crop or view_img:  # Add bbox to image
                                # Crop Box
                                BGR = True
                                xyxy_test = torch.tensor(xyxy).view(-1, 4)
                                b = xyxy2xywh(xyxy_test)  # boxes
                                b[:, 2:] = b[:, 2:] * 1.02 + \
                                    10  # box wh * gain + pad
                                xyxy_crop = xywh2xyxy(b).long()
                                clip_coords(xyxy_crop, imc.shape)
                                crop = imc[int(xyxy_crop[0, 1]):int(xyxy_crop[0, 3]), int(
                                    xyxy_crop[0, 0]):int(xyxy_crop[0, 2]), ::(1 if BGR else -1)]
                                # End crop
                                # To classification

                                output_brand, sc1 = objectClasssifyInfer.predict(
                                    chinh_model, crop, return_features=False)

                                result_text_spotting = textClassifyInfer.spotting_text(
                                    pan_detect, craft_detect, mmocr_recog, crop)

                                result = textClassifyInfer.predict(result_text_spotting.copy(
                                ), classifyModel_level1, classifyModel_level3=None, branch=True)

                                branch_0 = result[-1][0][0].replace(" ", "_")
                                text_list = []
                                for i in result[:-1]:
                                    text = i['text'].lower().replace(' ', '_')
                                    text_list.append(text)
                                test_keyword = False
                                for text_ in text_list:
                                    if text_ in keywords:
                                        test_keyword = True
                                        break
                                c = (list(labels_branch.keys())
                                     [output_brand].strip())
                                if test_keyword == True:
                                    output_final_branch = result[-1][0][0]
                                elif len(text_list) == 0 and sc1 < 0.98:
                                    output_final_branch = 'Unknow'
                                elif len(text_list) >= 4:
                                    branch_0 = c
                                    if sc1 > 0.95:
                                        output_final_branch = c
                                    else:
                                        output_final_branch = 'Unknow'
                                else:
                                    if sc1 > 0.93:
                                        output_final_branch = c
                                    else:
                                        output_final_branch = 'Unknow'
                                label = output_final_branch
                                if len(text_list) >= 2 and label != 'Unknow':
                                    check_list = False
                                    output_merge, _ = objectClasssifyInfer.predict_merge_model(
                                        model_step, crop)
                                    if len(dict_middle[str(output_merge)]) > 1:
                                        check_list = True
                                    name_merge = dict_middle[str(
                                        output_merge)][-1]
                                    brand_merge = name_merge.split("/")[0]
                                    # print(
                                    #     "============RESULT CHINH====================")
                                    # print(name_merge)

                                    temp_step = None
                                    if output_final_branch in ["f99foods", "heinz", "bubs_australia", "megmilksnowbrand", "meiji"]:
                                        pass
                                    else:
                                        if output_final_branch in mapping_checkpoints.keys():
                                            path_checkpoint = "/content/milk_classification/classifyText/textClassify/checkpoints/product/brands/" + \
                                                mapping_checkpoints[output_final_branch]
                                            classifyModel_level3 = textClassifyInfer.load_model(
                                                model_3=True, path_checkpoint=path_checkpoint)
                                            result_2 = textClassifyInfer.predict(
                                                result_text_spotting, classifyModel_level1, classifyModel_level3, step=True, added_text=''.replace(' ', '_'))
                                            temp_step = result_2[-1][0].replace(
                                                " ", "_")
                                            brand_text = meger_label_branch(
                                                labels_end, 2, temp_step)
                                            # print(
                                            #     "============RESULT THANH====================")
                                            # print(
                                            #     result_2[-1][0].replace(" ", "_"))
                                        else:
                                            print(output_final_branch)

                                    esem = Ensemble(
                                        output_final_branch, output_merge, temp_step, dict_middle, dict_step, text_list)
                                    label = esem.run()

                                # print("============TEXT===============")
                                # print(text_list)
                                # print("============RESULTS END===============")
                                # print(label)
                                # display_image(crop)
                                file_label.writelines(
                                    "=================== NHÃN SẢN PHẨM ====================\n")
                                file_label.writelines(label + "\n")
                                im0 = plot_one_box(xyxy, im0, label=label, color=colors(
                                    0, True), line_width=line_thickness)
                        result_text = textSpottingInfer.predict(
                            imc, craft_detect, model_recognition)
                        final_res = word2line(result_text, imc)
                        list_text = []
                        for res in final_res:
                            x1, y1, w, h = res['box']
                            # bbox = res['boxes']
                            # res_b = textSpottingInfer.get_box_from_poly(bbox)
                            x2 = x1+w
                            y2 = y1+h
                            # c1, c2 = (int(res_b[0]), int(res_b[1])), (int(res_b[2]), int(res_b[3]))
                            text = res['text']
                            # tokens = text.split()
                            # text_end = ''
                            # for token in tokens:
                            #     text_end=text_end+spell.correction(token)+" "
                            list_text.append((text))
                            c1, c2 = (x1, y1), (x2, y2)
                            cv2.rectangle(im0, c1, c2, colors(
                                0, True), thickness=line_thickness, lineType=cv2.LINE_AA)
                        list_text = [x+'\n' for x in list_text]

                        file_label.writelines(
                            "=================== NỘI DUNG TEXT ====================\n")
                        file_label.writelines(list_text)
                    # End model recognition
                    # Save results (image with detections)

                    if save_img:
                        if dataset.mode == 'image':
                            cv2.imwrite(save_path, im0)
                        else:  # 'video' or 'stream'
                            if vid_path[i] != save_path:  # new video
                                vid_path[i] = save_path
                                if isinstance(vid_writer[i], cv2.VideoWriter):
                                    # release previous video writer
                                    vid_writer[i].release()
                                if vid_cap:  # video
                                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                                    w = int(vid_cap.get(
                                        cv2.CAP_PROP_FRAME_WIDTH))
                                    h = int(vid_cap.get(
                                        cv2.CAP_PROP_FRAME_HEIGHT))
                                else:  # stream
                                    fps, w, h = 30, im0.shape[1], im0.shape[0]
                                    save_path += '.mp4'
                                vid_writer[i] = cv2.VideoWriter(
                                    save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                            vid_writer[i].write(im0)


In [ ]:
run(
    source='/content/drive/MyDrive/Colab Notebooks/data/Nhận diện từng sản phẩm sữa',
    project='/content/drive/MyDrive/Colab Notebooks/results',
    name = 'Nhận diện từng sản phẩm sữa'
)

YOLOv5 🚀 2021-9-9 torch 1.7.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7062001 parameters, 0 gradients
Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients


image 1/1019 /content/drive/MyDrive/Colab Notebooks/data/Nhận diện từng sản phẩm sữa/01.03(1).jpg: 

/content/milk_classification/textSpotting/CRAFT-pytorch/inference.py:88: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if boxes != []:
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/ngram.py:193: DeprecationWarning: Method ngram deprecated, use method split with pad_len=0 instead.
  warnings.warn('Method ngram deprecated, use method split with pad_len=0 instead.', DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid 

image 2/1019 /content/drive/MyDrive/Colab Notebooks/data/Nhận diện từng sản phẩm sữa/01.03.jpg: 

/content/milk_classification/textSpotting/CRAFT-pytorch/inference.py:88: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if boxes != []:
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/ngram.py:193: DeprecationWarning: Method ngram deprecated, use method split with pad_len=0 instead.
  warnings.warn('Method ngram deprecated, use method split with pad_len=0 instead.', DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid 

...

/content/milk_classification/textSpotting/CRAFT-pytorch/inference.py:88: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if boxes != []:
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/ngram.py:193: DeprecationWarning: Method ngram deprecated, use method split with pad_len=0 instead.
  warnings.warn('Method ngram deprecated, use method split with pad_len=0 instead.', DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid 

image 805/1019 /content/drive/MyDrive/Colab Notebooks/data/Nhận diện từng sản phẩm sữa/226735206_4214206288664312_9026775391398201067_n.png: 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/ngram.py:193: DeprecationWarning: Method ngram deprecated, use method split with pad_len=0 instead.
  warnings.warn('Method ngram deprecated, use method split with pad_len=0 instead.', DeprecationWarning)
/content/milk_classification/textSpotting/CRAFT-pytorch/inference.py:88: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if boxes != []:
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid 

image 806/1019 /content/drive/MyDrive/Colab Notebooks/data/Nhận diện từng sản phẩm sữa/226895267_4307497009302024_6575530179393374500_n.jpg: 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/ngram.py:193: DeprecationWarning: Method ngram deprecated, use method split with pad_len=0 instead.
  warnings.warn('Method ngram deprecated, use method split with pad_len=0 instead.', DeprecationWarning)
/content/milk_classification/textSpotting/CRAFT-pytorch/inference.py:88: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if boxes != []:


In [14]:
%cd /content/milk_classification

/content/milk_classification


## Crop image test

In [ ]:
def run(weights=['models/weights/binh_new_best.pt','models/weights/sua_new_best.pt'],  # model.pt path(s)
        source='/content/drive/MyDrive/Colab Notebooks/data_test_end',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='0',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=True,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project='/content/drive/MyDrive/Colab Notebooks/results',  # save results to project/name
        name='data_test_end',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=2,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        ):
  
  save_img = not nosave and not source.endswith('.txt')  # save inference images
  webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
      ('rtsp://', 'rtmp://', 'http://', 'https://'))

  # Directories
  save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
  (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

  # Initialize
  set_logging()
  device = select_device(device)
  half &= device.type != 'cpu'  # half precision only supported on CUDA

  # Load model
  w = weights[0] if isinstance(weights, list) else weights
  classify, suffix = False, Path(w).suffix.lower()
  pt, onnx, tflite, pb, graph_def = (suffix == x for x in ['.pt', '.onnx', '.tflite', '.pb', ''])  # backend
  stride, names = 64, [f'class{i}' for i in range(1000)]  # assign defaults
  if pt:
      model_binh_sua = attempt_load(weights[0], map_location=device)  # load FP32 model
      model_sua = attempt_load(weights[1], map_location=device)  # load FP32 model

      stride_binh = int(model_binh_sua.stride.max())  # model stride
      names_binh = model_binh_sua.module.names if hasattr(model_binh_sua, 'module') else model_binh_sua.names  # get class names
      names_sua = model_sua.module.names if hasattr(model_sua, 'module') else model_sua.names  # get class names
      if half:
          model_binh_sua.half()  # to FP1
          names_sua.half()  # to FP1
      if classify:  # second-stage classifier
          model_binh_sua_c = load_classifier(name='resnet50', n=2)  # initialize
          model_binh_sua_c.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()

          model_sua_c = load_classifier(name='resnet50', n=2)  # initialize
          model_sua_c.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()
  elif onnx:
      check_requirements(('onnx', 'onnxruntime'))
      import onnxruntime
      session = onnxruntime.InferenceSession(w, None)
  imgsz_binh = check_img_size(imgsz, s=stride_binh)  # check image size

  # Dataloader
  if webcam:
      view_img = check_imshow()
      cudnn.benchmark = True  # set True to speed up constant image size inference
      dataset = LoadStreams(source, img_size=imgsz_binh, stride=stride_binh)
      bs = len(dataset)  # batch_size
  else:
      dataset = LoadImages(source, img_size=imgsz_binh, stride=stride_binh)
      bs = 1  # batch_size
  vid_path, vid_writer = [None] * bs, [None] * bs

  # Run inference
  if pt and device.type != 'cpu':
      model_binh_sua(torch.zeros(1, 3, imgsz_binh, imgsz_binh).to(device).type_as(next(model_binh_sua.parameters())))  # run once
      model_sua(torch.zeros(1, 3, imgsz_binh, imgsz_binh).to(device).type_as(next(model_sua.parameters())))  # run once
  t0 = time.time()
  for path, img, im0s, vid_cap in dataset:
      if pt:
          img = torch.from_numpy(img).to(device)
          img = img.half() if half else img.float()  # uint8 to fp16/32
      elif onnx:
          img = img.astype('float32')
      img /= 255.0  # 0 - 255 to 0.0 - 1.0
      if len(img.shape) == 3:
          img = img[None]  # expand for batch dim

      # Inference
      t1 = time_sync()
      if pt:
          visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
          pred_binh = model_binh_sua(img, augment=augment, visualize=visualize)[0]
          pred_sua = model_sua(img, augment=augment, visualize=visualize)[0]

      # NMS
      pred_binh = non_max_suppression(pred_binh, 0.4, iou_thres, classes, agnostic_nms, max_det=max_det)
      pred_sua = non_max_suppression(pred_sua, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
      t2 = time_sync()

      # Second-stage classifier (optional)
      if classify:
          pred_binh = apply_classifier(pred_binh, model_binh_sua_c, img, im0s)
          pred_sua = apply_classifier(pred_sua, model_sua_c, img, im0s)
      
      # Process predictions binh sua
      
      for i, (det_binh,det_sua) in enumerate(zip(pred_binh,pred_sua)):  # detections per image
        if webcam:  # batch_size >= 1
            p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
        else:
            p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

        p = Path(p)  # to Path
        save_path = str(save_dir / p.name)  # img.jpg
        txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
        s += '%gx%g ' % img.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        imc = im0.copy()  # for save_crop
        if len(det_sua):
          # Rescale boxes from img_size to im0 size
          det_sua[:, :4] = scale_coords(img.shape[2:], det_sua[:, :4], im0.shape).round()

          # Print results
          for c in det_sua[:, -1].unique():
              n = (det_sua[:, -1] == c).sum()  # detections per class
              s += f"{n} {names_sua[int(c)]}{'s' * (n > 1)}, "  # add to string
          # Write results
          stt = 0
          for *xyxy, conf, cls in reversed(det_sua):
            name = "{0}_return.txt".format(stt)
            txt_path_end = save_path.replace('.jpg',name).replace('.png',name)
            image_path = txt_path_end.replace(".txt",".jpg")
            with open(txt_path_end,"a") as file_txt:
              if save_img or save_crop or view_img:  # Add bbox to image
                # Crop Box
                BGR = True
                xyxy_test = torch.tensor(xyxy).view(-1, 4)
                b = xyxy2xywh(xyxy_test)  # boxes
                b[:, 2:] = b[:, 2:] * 1.02 + 10  # box wh * gain + pad
                xyxy_crop = xywh2xyxy(b).long()
                clip_coords(xyxy_crop, imc.shape)
                crop = imc[int(xyxy_crop[0, 1]):int(xyxy_crop[0, 3]), int(xyxy_crop[0, 0]):int(xyxy_crop[0, 2]), ::(1 if BGR else -1)]
                # End crop
                # To classification
                cv2.imwrite(image_path,crop)
                features,[output_brand,sc1] = objectClasssifyInfer.predict(chinh_model,crop,return_features=True)
                result_text_spotting = textClassifyInfer.spotting_text(pan_detect,craft_detect,mmocr_recog,crop)
                result = textClassifyInfer.predict(result_text_spotting.copy(),classifyModel_level1,classifyModel_level3,branch=True)
                branch_0 = result[-1][0][0].replace(" ","_")
                text_list = []
                for i in result[:-1]:
                    text = i['text'].lower().replace(' ','_')
                    text_list.append( text+"\n")
                test_keyword = False
                for text_ in text_list:
                    if text_ in keywords:
                        test_keyword = True 
                        break 
                c = (list(labels_branch.keys())[output_brand].strip())
                if test_keyword ==True:
                  output_final_branch = result[-1][0][0]
                elif len(text_list) == 0 and sc1 < 0.98:
                  output_final_branch = 'Unknow'
                elif len(text_list) >=4:
                  branch_0 = c
                  if sc1 > 0.95:
                    output_final_branch = c
                  else:
                    output_final_branch = 'Unknow'
                else:
                  if sc1 > 0.93:
                    output_final_branch = c
                  else:
                    output_final_branch = 'Unknow'
                label = output_final_branch
                text_list.append(label+"\n")
                # if len(text_list) >= 2 and label!='Unknow':
                #   result_2 = textClassifyInfer.predict(result_text_spotting,classifyModel_level1,classifyModel_level3,step=True,added_text='',features=features)
                # label = label + " | " + result_2[-1][0].replace(" ","_")
                # text_list.append(label+"\n")
                file_txt.writelines(text_list)
              stt+=1

In [ ]:
run()